# Introduction
多変量解析TMVA講習で学んだように、Machine Learningについての知識は、実験データを解析する上で欠かせないものになりつつあります。特に近年Deep learning(深層学習)が急速に発展してきており、実験物理分野でも用いられるようになりました。

Deep learningを扱うツールが整備されたことにより、誰でも簡単にDeep learningが使えるようになりました。一方で、その背後にある仕組みを知らないまま扱うと、落とし穴にハマることがあるかもしれません。

この講義では、Deep learningの基礎、特にDeep learningのベースとなるパーセプトロンを学んでいきます。式を追うだけでは理解しきれないことも多いと思うので、サンプルコードをぜひ自分の手で動かしてみてください。

## Jupyter notebookについて
この講義ではjupyter notebookを提供します。
数式と文章で解説をし、要所要所で式やアルゴリズムを理解するコードがpythonで書かれています。

## パッケージのインポート
まず始めに、必要なパッケージをインポートします。
numpyは多次元配列を効率的に行うパッケージで、matplotlibはプロット等を行うパッケージです。
"%matplotlib inline"はjupyter notebookでプロットを行う際に必要ですが、通常のpython scriptとしてコードを実行する際は必要ありません。

In [ ]:
#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# パーセプトロン (Perceptron)
(単純)パーセプトロンは1957年に開発されたアルゴリズムで、分類問題を解くことができます。

分類問題とは、入力を複数のクラスにラベル付けすることです。物理実験分野ではシグナルとバックグラウンドの分類としてよく用いられます。

例えば、n個の入力変数$ \mathbf{x}=\{x_1, x_2,\dots,x_n\} $からクラス$ t\in\{0, 1\} $を推定する分類問題を考えると、入力がシグナル由来のものならば
$$ t = y(\mathbf{x}_S) = 1 $$
入力がバックグラウンド由来のものならば
$$t = y(\mathbf{x}_B) = 0 $$
となるような関数$y(\mathbf{x})$を作ることが目的になります。

パーセプトロンでは、関数を以下のように定義します。
$$
\begin{align*}
y(\mathbf{x}|\mathbf{w}, b) &= \Gamma(\mathbf{w}\cdot \mathbf{x} + b) \\
\Gamma(a) &= \begin{cases}
    1 & (a\geq0)\\
    0 & (a<0)
  \end{cases}
\end{align*}
$$
ここで、$ \mathbf{w},b $は関数のパラメータ、$ \Gamma $はステップ関数です。関数のパラメータを問題によって調整することで、分類問題を解くことができます。

## ANDゲート
まず初めに2変数( $x_1, x_2$ )を入力とするパーセプトロンを用いてANDゲートを実装してみましょう。

ANDゲートは
             
| $x_1$ | $x_2$ | AND |
| :-: | :-: | :-: |
| 0 | 0 | 0 |
| 0 | 1 | 0 |
| 1 | 0 | 0 |
| 1 | 1 | 1 |

のように、2つの入力変数がともに1のときのみ1を返し、それ以外のときは0を返すような関数です。
入力変数が2つなので、パーセプトロン関数を明示的に書くと、
$$
\begin{aligned}
y(x_1,x_2|w_1,w_2, b) &= \Gamma(w_1 x_1 + w_2 x_2 + b) \\
\Gamma(a) &= \begin{cases}
    1 & (a\geq0)\\
    0 & (a<0)
  \end{cases}
\end{aligned}
$$
となります。これをpythonで書くと以下のようになります。

In [ ]:
def and_gate(x1, x2):
    w1 = 1.0  # FIXME: 適切なパラメータの値を入力してください
    w2 = 2.0  # FIXME: 適切なパラメータの値を入力してください
    b = 3.0  # FIXME: 適切なパラメータの値を入力してください
    a = w1 * x1 + w2 * x2 + b
    if a >= 0.:
        return 1
    else:
        return 0


# 正しく実装できているかを出力
print("AND Gate:")
print("  x1, x2, y")
print("   0,  0, ", and_gate(0, 0), "Correct" if and_gate(0, 0) == 0 else "Wrong")  # 正解は 0
print("   0,  1, ", and_gate(0, 1), "Correct" if and_gate(0, 1) == 0 else "Wrong")  # 正解は 0
print("   1,  0, ", and_gate(1, 0), "Correct" if and_gate(1, 0) == 0 else "Wrong")  # 正解は 0
print("   1,  1, ", and_gate(1, 1), "Correct" if and_gate(1, 1) == 1 else "Wrong")  # 正解は 1

上の例ではパラメータが適当なため、正しいANDゲートにはなっていません。パラメータ($w_1, w_2, b$)をどのように設定すれば、この関数がANDゲートとなるでしょうか？

実は答えは一意には決まらず無数にあるのですが、例えば$(w_1, w_2, b) = (1, 1, -1.5)$とするとうまくいきます。確認してみてください。

パーセプトロンの値をプロットすると
<img src="jupyterFigure/AND.png" width="400">
となります。
赤い領域で$y=1$、青い領域で$y=0$を出力しています。
パーセプトロンの式の形からもわかるように、$(x_1, x_2)$平面上を直線($w_1 x_1 + w_2 x_2 + b=0$)で区切ったような出力となります。

## NANDゲート・ORゲート
NANDゲートやORゲートは

| $x_1$ | $x_2$ | NAND | OR |
| -- | -- | -- | -- |
| 0 | 0 | 1 | 0 |
| 0 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 1 |

のような関数です。各自どのようにパラメータを決めればよいか確認してみてください。


In [ ]:
def nand_gate(x1, x2):
    w1 = 1.0  # FIXME: 適切なパラメータの値を入力してください
    w2 = 2.0  # FIXME: 適切なパラメータの値を入力してください
    b = 3.0  # FIXME: 適切なパラメータの値を入力してください
    a = w1 * x1 + w2 * x2 + b
    if a >= 0.:
        return 1
    else:
        return 0


def or_gate(x1, x2):
    w1 = 1.0  # FIXME: 適切なパラメータの値を入力してください
    w2 = 2.0  # FIXME: 適切なパラメータの値を入力してください
    b = 3.0  # FIXME: 適切なパラメータの値を入力してください
    a = w1 * x1 + w2 * x2 + b
    if a >= 0.:
        return 1
    else:
        return 0


# 正しく実装できているかを出力
print("NAND Gate:")
print("  x1, x2, y")
print("   0,  0, ", nand_gate(0, 0), "Correct" if nand_gate(0, 0) == 1 else "Wrong")  # 正解は 1
print("   0,  1, ", nand_gate(0, 1), "Correct" if nand_gate(0, 1) == 1 else "Wrong")  # 正解は 1
print("   1,  0, ", nand_gate(1, 0), "Correct" if nand_gate(1, 0) == 1 else "Wrong")  # 正解は 1
print("   1,  1, ", nand_gate(1, 1), "Correct" if nand_gate(1, 1) == 0 else "Wrong")  # 正解は 0

print("OR Gate:")
print("  x1, x2, y")
print("   0,  0, ", or_gate(0, 0), "Correct" if or_gate(0, 0) == 0 else "Wrong")  # 正解は 0
print("   0,  1, ", or_gate(0, 1), "Correct" if or_gate(0, 1) == 1 else "Wrong")  # 正解は 1
print("   1,  0, ", or_gate(1, 0), "Correct" if or_gate(1, 0) == 1 else "Wrong")  # 正解は 1
print("   1,  1, ", or_gate(1, 1), "Correct" if or_gate(1, 1) == 1 else "Wrong")  # 正解は 1


## ロジスティック回帰 (Logistic regression)
これまで見てきた３つのゲート(AND,OR,NAND)は、出力が一意に定まる問題でした。一方で現実の問題のほとんどは一意にラベル付けはできません。例えば($H\rightarrow\gamma\gamma$)のように、観測された事象がシグナルであるのかバックグラウンドであるのかは確率的にしか評価できないことがほとんどです。
<img src="jupyterFigure/Hgg.png" width="300">
そのため、関数の出力を0 or 1とするのではなく、0から1まで連続的な値を取るように変更します。ここで出力は、クラスのラベル(S or B)ではなく、そのラベルを取る確率を表します。
出力として0 or 1を取るステップ関数の代わりによく使われる関数がシグモイド関数です。
$$ \sigma(x) = \frac{1}{1+e^{-x}} $$
<img src="jupyterFigure/sigmoid.png" width="400">

$x\rightarrow \pm \infty$の極限では、ステップ関数もシグモイド関数も同じ値となりますが、x=0付近で、シグモイド関数はなめらかになっており、0から1まで連続的な値を取ることができます。また、x軸のスケールを変更すると、シグモイド関数はステップ関数に限りなく近づくという性質もあります。

パーセプトロンの式は変更されて、
$$
\begin{align*}
y(\mathbf{x}|\mathbf{w}, b) &= \sigma(\mathbf{w}\cdot \mathbf{x} + b) \\
\sigma(x) &= \frac{1}{1+e^{-x}}
\end{align*}
$$
となります。先程の式のステップ関数の部分がシグモイド関数に変わっています。
この関数を用いて分類・回帰をすることをロジスティック回帰(Logistic regression)と呼ぶこともありますし、パーセプトロンと呼ぶこともあります。ここではパーセプトロンと呼びます。

なお、このステップ関数やシグモイド関数に対応する部分を__活性化関数(activation function)__とも呼びます。


## 実例
この式を用いて、実際に問題を解いてみましょう。

次のセルはヘルパー関数です。データ点の作成や、プロットを行います。内容は完全に理解する必要はありませんが、もし余力があればコードを追ってみてください。

In [ ]:
# 中心値の異なる2つの二次元ガウス分布
def get_dataset_1():
    from numpy.random import default_rng
    rng = default_rng(seed=0)  # 今回はデータセットの乱数を固定させます。

    num_signal = 100  # 生成するシグナルイベントの数
    num_background = 100  # 生成するバックグラウンドイベントの数

    # データ点の生成
    ## 平均(x1,x2) = (1.0, 0.0)、分散=1の２次元ガウス分布
    x_sig = rng.multivariate_normal(mean=[1.0, 0.0],
                                    cov=[[1., 0.], [0., 1.]],
                                    size=num_signal)
    t_sig = np.ones((num_signal, 1))  # Signalは1にラベリング

    ## 平均(x1,x2) = (-1.0, 0.0)、分散=1の２次元ガウス分布
    x_bg = rng.multivariate_normal(mean=[-1.0, 0.0],
                                   cov=[[1., 0.], [0., 1.]],
                                   size=num_background)
    t_bg = np.zeros((num_background, 1))  # Backgroundは0にラベリング

    # 2つのラベルを持つ学習データを1つにまとめる
    x = np.concatenate([x_sig, x_bg])
    t = np.concatenate([t_sig, t_bg])

    # データをランダムに並び替える
    p = rng.permutation(len(x))
    x, t = x[p], t[p]

    return x, t


# 二次元ガウス分布と一様分布
def get_dataset_2():
    from numpy.random import default_rng
    rng = default_rng(seed=0)  # 今回はデータセットの乱数を固定させます。

    num_signal = 100  # 生成するシグナルイベントの数
    num_background = 1000  # 生成するバックグラウンドイベントの数

    # データ点の生成
    ## 平均(x1,x2) = (1.0, 0.0)、分散=1の２次元ガウス分布
    x_sig = rng.multivariate_normal(mean=[1.0, 0],
                                    cov=[[1, 0], [0, 1]],
                                    size=num_signal)
    t_sig = np.ones((num_signal, 1))  # Signalは1にラベリング

    ## (-5, +5)の一様分布
    x_bg = rng.uniform(low=-5, high=5, size=(num_background, 2))
    t_bg = np.zeros((num_background, 1))  # Backgroundは0にラベリング

    # 2つのラベルを持つ学習データを1つにまとめる
    x = np.concatenate([x_sig, x_bg])
    t = np.concatenate([t_sig, t_bg])

    # データをランダムに並び替える
    p = rng.permutation(len(x))
    x, t = x[p], t[p]

    return x, t


# ラベル t={0,1}を持つデータ点のプロット
def plot_datapoint(x, t):
    # シグナル/バックグラウンドの抽出
    xS = x[t[:, 0] == 1]  # シグナルのラベルだけを抽出
    xB = x[t[:, 0] == 0]  # バックグラウンドのラベルだけを抽出

    # プロット
    plt.scatter(xS[:, 0], xS[:, 1],label='Signal', c='red', s=10)  # シグナルをプロット
    plt.scatter(xB[:, 0], xB[:, 1], label='Background', c='blue', s=10)  # バックグラウンドをプロット
    plt.xlabel('x1')  # x軸ラベルの設定
    plt.ylabel('x2')  # y軸ラベルの設定
    plt.legend()  # legendの表示
    plt.show()


# prediction関数 の等高線プロット (fill)
def plot_prediction(prediction, *args):
    # 等高線を描くためのメッシュの生成
    x1, x2 = np.mgrid[-5:5:100j, -5:5:100j]  # x1 = (-5, 5), x2 = (-5, 5) の範囲で100点x100点のメッシュを作成
    x1 = x1.flatten()  # 二次元配列を一次元配列に変換 ( shape=(100, 100) => shape=(10000, ))
    x2 = x2.flatten()  # 二次元配列を一次元配列に変換 ( shape=(100, 100) => shape=(10000, ))
    x = np.array([x1, x2]).T

    #  関数predictionを使って入力xから出力yを計算し、等高線プロットを作成
    y = prediction(x, *args)
    cs = plt.tricontourf(x[:, 0], x[:, 1], y.flatten(), levels=10)
    plt.colorbar(cs)


# prediction関数 の等高線プロット (line)
def plot_prediction_line(prediction, *args):
    # 等高線を描くためのメッシュの生成
    x1, x2 = np.mgrid[-5:5:100j, -5:5:100j]  # x1 = (-5, 5), x2 = (-5, 5) の範囲で100点x100点のメッシュを作成
    x1 = x1.flatten()  # 二次元配列を一次元配列に変換 ( shape=(100, 100) => shape=(10000, ))
    x2 = x2.flatten()  # 二次元配列を一次元配列に変換 ( shape=(100, 100) => shape=(10000, ))
    x = np.array([x1, x2]).T

    #  関数predictionを使って入力xから出力yを計算し、等高線プロットを作成
    y = prediction(x, *args)
    plt.tricontour(x[:, 0], x[:, 1], y.flatten(), levels=10)


def plot_prediction_regression(x, t, prediction, w1, b1, w2, b2):
    def _sigmoid(x):
        return 1. / (1 + np.exp(-x))

    def _perceptron(x, w, b):
        a = np.dot(x, w) + b  # w・x + b
        return _sigmoid(a)

    #  データ点のプロット
    plt.scatter(x, t, s=10, c='black')

    #  関数predictionの出力をプロット
    y = prediction(x, w1, b1, w2, b2)
    plt.plot(x, y, c='red')

    # 中間層の各ノードの出力をプロット
    num_nodes = len(w2)
    for i in range(num_nodes):
        y = w2[i] * _perceptron(x, w1[:, i], b1[i])
        plt.plot(x, y, linestyle='dashed')  # (中間層のノードの出力 * 重み)をプロット
    plt.plot(x, np.full_like(x, b2[0]), linestyle='dashed')  # 最後の層のバイアスタームのプロット



上の関数を使ってデータ点をプロットしましょう。

In [ ]:
# データ点の取得
x, t = get_dataset_1()

# データ点をプロット
plot_datapoint(x, t)


このようなシグナル・バックグラウンドを分類することを考えます。この分類問題では、シグナルとバックグラウンドの分布が一部重なっています。したがって、そのような中間部分では出力0 or 1ではなく、そのイベントがシグナルである確率を返すように、シグモイド関数を使います。

パーセプトロンの式は

In [ ]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))


def perceptron(x1, x2, w1, w2, b):
    a = w1 * x1 + w2 * x2 + b
    return sigmoid(a)


となります。最後の部分だけがステップ関数からシグモイド関数に変わっています。
より一般的にnumpyのarrayを用いてベクトル的に書くと、

In [ ]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))


def perceptron(x, w, b):
    a = np.dot(x, w) + b  # w・x + b
    return sigmoid(a)


のようになります。ここで、x,wはベクトルで、np.dot(x,w)はxとwの内積を取る操作をします。

試しに、入力変数($x$)やパラメータ$(w,b)$を変化させてどのような値が返ってくるのか確認してみましょう。

In [ ]:
x = np.array([1.0, 2.0])
w = np.array([1.0, 1.0])
b = np.array(-1.5)

print(f"w1 * x1 + w2 * x2 + b = {w[0]} * {x[0]} + {w[1]} * {x[1]} + {b}")
print(f"                      = {perceptron(x, w, b)}")


$$
\begin{align}
\mathbf{w}\cdot \mathbf{x} + b &= 
\begin{pmatrix}
w_1 & w_2
\end{pmatrix}
\cdot
\begin{pmatrix}
x_1 \\
x_2
\end{pmatrix}
+ b \\
&=
\begin{pmatrix}
1.0 & 1.0
\end{pmatrix}
\cdot
\begin{pmatrix}
1.0 \\
2.0
\end{pmatrix}
-1.5  = 1.5 \\
\sigma(1.5) &= \frac{1}{1+e^{-1.5}} = 0.818
\end{align}
$$
を計算しています。

xには複数の入力を与えることもできます。xのshapeを(データセットの数, 変数の数)、wのshapeを(変数の数)とすれば、perceptronの式の np.dot(x, w) の返り値のshapeは(データセットの数, )となります。

例えば次のようにして、一回の計算で複数のデータ点を処理できます。

In [ ]:
x = np.array([
    [0.0, 0.0],  # event 1
    [0.0, 1.0],  # event 2
    [1.0, 0.0],  # event 3
    [1.0, 1.0],  # event 4
])
w = np.array([1.0, 1.0])
b = np.array(-1.5)

print(perceptron(x, w, b))  # 4イベントの出力がまとめて返される


先程定義したヘルパー関数を用いて等高線プロットを作ると、

In [ ]:
# パーセプトロンのパラメータ
w = np.array([1.0, 1.0])  # FIXME: パラメータの値を適当に変更してみましょう
b = np.array(1.5)  # FIXME: パラメータの値を適当に変更してみましょう

# パーセプトロンの出力を等高線プロット
plot_prediction(perceptron, w, b)

# データ点の取得
x, t = get_dataset_1()

# データ点をプロット
plot_datapoint(x, t)


となります。先程のステップ関数を使った例とは異なり、なめらかな出力をしていることがわかります。
パラメータの値を変化させて、パーセプトロンの出力がどのように変化するのか確かめてみましょう。
"最適な"パラメータの値は何になるでしょうか？

## パーセプトロンのグラフ表現
パーセプトロンの式をグラフで表すと視覚的にわかりやすく、また後々より複雑なモデルを作るときに、全体像を把握しやすくなります。
パーセプトロンの式は
$$
\begin{align*}
y(\mathbf{x}|\mathbf{w}, b) &= \sigma(\mathbf{w}\cdot \mathbf{x} + b)
\end{align*}
$$
でしたが、これをさらに分解して、各成分を明示的に書くと、
$$
\begin{align*}
a &= w_1 \cdot x_1 + w_2 \cdot x_2 + b \\
y &= \sigma(a)
\end{align*}
$$
となります。この式を
<img src="jupyterFigure/perceptron_graph.png" width="300">
のように表現します。グラフにすることによって、入力変数($(x_1, x_2)$)が右に伝搬して出力($y$)までどのように計算されるのかがわかりやすくなります。

## 機械学習
これまでは手で関数のパラメータを決めてきましたが、問題が複雑になると、限界がきます。データからモデルのパラメータを自動で決めるのが、機械学習です。適切に学習させることで、人間が手でチューニングしたものよりも良い結果を得ることができます。

さて、この学習はどのようにして進めたら良いでしょうか？
最良の出力を得るパラメータとはどのようなものでしょうか？

ここでまた2クラス分類問題を考えます。n個の入力変数($\mathbf{x}=\{x_1, x_2,\dots,x_n\}$)からクラス($t\in\{0, 1\}$)を推定します。

ある観測値($ \mathbf{x}=\{x_1, x_2,\dots,x_n\}$)が得られた時、それがシグナルである確率を$y(\mathbf{x}|\mathbf{w}, b)$としていました(バックグラウンドである確率は$1-y(\mathbf{x}|\mathbf{w}, b)$)。ある観測値の集合が与えられ、それらに対してのラベルがわかっている時、パラメータが観測値を説明する確率は
$$
p(\mathbf{t}|\mathbf{w},b ) = \prod_{n=1}^{N} y_n^{t_n}\left(1-y_n \right)^{1-t_n}
$$
となります。この式では、$t_n = 1$の時$p_n = y(\mathbf{x}_n)$、$t_n = 0$の時$p_n=1-y(\mathbf{x}_n)$となります。

最尤法の考え方から、この確率を最大にするモデルパラメータを決定すれば良いことがわかります。負の対数を取ることで、
$$
E(\mathbf{w}, b) = -\log p(\mathbf{t}|\mathbf{w}, b) = -\sum_{n=1}^{N} \left( t_n \log y_n + \left( 1-t_n \right) \log \left( 1-y_n \right) \right)
$$
をパラメータ($\mathbf{w}, b$)に対して最小化する問題に帰着します。
機械学習において最小化する関数のことを__誤差関数(loss function)__と呼びます。
また、上のような関数をこの式を__交差エントロピー誤差関数(Cross entropy error function)__と呼びます。
この例では誤差関数として交差エントロピー誤差関数を使うということになります。

このままでは学習に使うデータ数によって誤差関数のスケールが変わってしまうので、使用するデータ数で規格化したものを実際には使います。
$$
E(\mathbf{w}, b) = - \frac{1}{N} \sum_{n=1}^{N} \left( t_n \log y_n + \left( 1-t_n \right) \log \left( 1-y_n \right) \right)
$$

ここでは2クラス分類を考えましたが、多クラス分類でも同様に考えることができます。
K個のクラス($t\in\{0, 1, \dots, K\}$)に分類するときは、尤度関数はn番目のイベントの出力のクラス$k$に対する出力
$$
p(\mathbf{t}|\mathbf{w},b ) = \prod_{n=1}^{N} \prod_{k=1}^{K} y_{nk}^{t_{nk}}
$$
となります。ここで$t_{nk}$はn番目のイベントのラベルで、クラスkの時に1,他のクラスのときに0となります(すなわち$\sum_{k=1}^{K}t_{nk}=1$です)。$y_{nk}$はn番目のイベントのラベルkに対するパーセプトロンの出力です。
同様に式変形すると、
$$
E(\mathbf{w}, b) = -\frac{1}{N} \sum_{n=1}^{N}\sum_{k=1}^{K} t_{nk} \log y_{nk}
$$
となります。この式も同様に交差エントロピー誤差関数(Cross entropy error function)と呼びます。


### 最急降下法 (Gradient descent)
パーセプトロンのパラメータを最適にする指針(誤差関数)は決めましたが、その最小化方法は自明ではありません。
よく用いられる手法は勾配法です。勾配法は、その名の通り、関数の勾配を用いて、その極小値を探索する手法です。
ここでは勾配法の最もシンプルな例である最急降下法(Gradient descent)を用いて誤差関数を最小化します。

最急降下法では、入力が$\mathbf{x}=(x_1,\dots,x_n)$の関数$f(\mathbf{x})$の最小値を、以下のように逐次的に求めます。
$$
\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} - \epsilon \cdot \left. \frac{\partial f}{\partial \mathbf{x}} \right|_{\mathbf{x}=\mathbf{x}^{(k)}}
$$
ここで$\epsilon$は1回あたりの更新の大きさを決めるパラメータで、値が小さいと、ゆっくり最小値を探します。
関数の一次微分($\left. \frac{\partial f}{\partial \mathbf{x}} \right|_{\mathbf{x}=\mathbf{x}^{(k)}}$)は点$\mathbf{x}^{(k)}$での傾きを表しており、(適切な$\epsilon$を選べば)必ず関数の値が小さくなるように更新されます。

簡単な例でこのアルゴリズムが正しく動作することを確かめてみます。
$f(x_1,x_2)=x_1^2 + x_2^2$を最小化する$(x_1, x_2)$を求めます。更新式は
$$
\begin{align*}
x_0 &\leftarrow x_0 - \epsilon \cdot (2\cdot x_1) \\
x_1 &\leftarrow x_1 - \epsilon \cdot (2\cdot x_2) \\
\end{align*}
$$
となります。

python codeでは

In [ ]:
# 初期値の設定
x1 = 4.0
x2 = 3.0

learning_rate = 0.1  # ステップ幅
num_steps = 10  # 繰り返し回数
for _ in range(num_steps):
    # 値の更新
    x1 = x1 - learning_rate * (2 * x1)
    x2 = x2 - learning_rate * (2 * x2)


のようにして値を更新していきます。learning_rateが$\epsilon$に対応するパラメータです。

これを実行し、パラメータ($x_1, x_2$)の値の推移をプロットしてみましょう。

In [ ]:
# 目的関数の等高線プロット
x1, x2 = np.mgrid[-5:5:100j, -5:5:100j]  # x1 = (-5, 5), x2 = (-5, 5) の範囲で100点x100点のメッシュを作成
y = np.square(x1) + np.square(x2)  # y = x1^2 + x2^2
plt.contour(x1, x2, y, linestyles='dashed', levels=5)

# 初期値の設定
x1 = 4.0
x2 = 3.0

plt.scatter(x1, x2, c='black')  # 初期値のプロット

# 最急降下法の実行
learning_rate = 0.1  # ステップ幅、 FIXME: ステップ幅を適当に変更してみましょう
num_steps = 10  # 繰り返し回数、 FIXME: 繰り返し回数を適当に変更してみましょう
for _ in range(num_steps):
    # 値の更新
    x1 = x1 - 2 * x1 * learning_rate
    x2 = x2 - 2 * x2 * learning_rate

    plt.scatter(x1, x2, edgecolors='black', facecolor='None')  # 更新値のプロット

plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim([-5, 5])
plt.ylim([-5, 5])
plt.show()


初期値(黒丸)から10回分値を更新しています。最小値を取る$(x_1, x_2)=(0, 0)$に近づいていることがわかります。
ステップ幅や更新回数を変更して変化を確認してみてください。
特にステップ幅の大きさはどの程度が適切となるでしょうか？

### 最急降下法を用いたパーセプトロンの学習
それではパーセプトロンのパラメータ($\mathbf{w}, b$)を交差エントロピー
$$
\begin{align*}
E(\mathbf{w}, b) &= \frac{1}{N} \sum_{n=1}^{N} E_n = -\frac{1}{N} \sum_{n=1}^{N} \left( t_n \log y_n + \left( 1-t_n \right) \log \left( 1-y_n \right) \right) \\
y(\mathbf{x}_n|\mathbf{w}, b) &= \sigma(\mathbf{w}\cdot \mathbf{x}_n + b)
\end{align*}
$$
を目的関数として、最急降下法を用いて最適化してみましょう。

誤差関数の微分は、
$$
\begin{align*}
\frac{\partial E_n}{\partial \mathbf{w}}
&= \frac{\partial E_n}{\partial  y_n}\cdot \frac{\partial  y_n}{\partial a} \cdot \frac{\partial  a}{\partial \mathbf{w}}\\
&= \left[ \frac{y_n-t_n}{y_n(1-y_n)}\right] \cdot \left[ y_n(1-y_n) \right] \cdot \left[ \mathbf{x_n} \right] \\
&= \left(y_n-t_n\right) \cdot \mathbf{x_n} \\
\frac{\partial E_n}{\partial b}
&= \left(y_n-t_n\right)
\end{align*}
$$
となります。ぜひ各自手で計算してみてください。
($\frac{\partial \sigma(x)}{\partial x} = \sigma \cdot (1 - \sigma)$となることを使いました。)

一次微分の式を使って最急降下法で誤差関数を最小化します。
更新式をpythonで書くと以下のようになります。

In [ ]:
# データ点の取得
x, t = get_dataset_1()

# 初期値の設定
w = np.array([2.0, 0.0])
b = np.array(0.1)

# 最急降下法での学習
learning_rate = 0.1  # ステップ幅
num_steps = 1000  # 繰り返し回数
for _ in range(num_steps):
    y = perceptron(x, w, b)  # パーセプトロンの出力
    y = y[:, np.newaxis]  # numpy array の shape を t に合わせる

    grad_w = (y - t) * x
    grad_b = (y - t) * 1

    w = w - learning_rate * grad_w.mean(axis=0)  # 全てのデータ点についての平均値を取る
    b = b - learning_rate * grad_b.mean(axis=0)  # 全てのデータ点についての平均値を取る

# プロット
plot_prediction(perceptron, w, b)

## データ点をプロット
plot_datapoint(x, t)


## 単純パーセプトロンの限界
ここまで単純パーセプトロンが分類問題に有用であることを見てきました。しかし単純パーセプトロンは特定の問題においてしか有用でないことが知られています。例えば先程AND/OR/NAND回路を見てきましたが、XOR回路においてはどうなるでしょうか？

XOR回路は次のような真理値となります。

| $x_1$ | $x_2$ | XOR |
| -- | -- | -- |
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

これを単純パーセプトロン
$$
\begin{align*}
y(x_1,x_2|w_1,w_2, b) &= \Gamma(w_1 x_1 + w_2 x_2 + b) \\
\Gamma(a) &= \begin{cases}
    1 & (a\geq0)\\
    0 & (a<0)
  \end{cases}
\end{align*}
$$
で分類できるでしょうか？ 実はこの問題は分類ができません。式の形からわかるようにパーセプトロンでは線形に分類できる問題にしか対応できません。

もう一つの例として、一様分布するバックグラウンド上に、シグナルが二次元ガウス分布する状況を考えましょう。

In [ ]:
# データ点の取得 (x: 入力データ, t: ラベル)
x, t = get_dataset_1()

# データ点をプロット
plot_datapoint(x, t)


このような例を、単純パーセプトロンで分類させようとしても、適切な予測ができません。

In [ ]:
# データ点の取得
x, t = get_dataset_2()

# 初期値の設定
w = np.array([0.1, 0.0])
b = np.array(0.0)

# 最急降下法での学習
learning_rate = 0.1  # ステップ幅
num_steps = 1000  # 繰り返し回数
for _ in range(num_steps):
    y = perceptron(x, w, b)  # パーセプトロンの出力
    y = y[:, np.newaxis]  # numpy array の shape を t に合わせる

    grad_w = (y - t) * x
    grad_b = (y - t) * 1

    w = w - learning_rate * grad_w.mean(axis=0)  # 全てのデータセットについての平均値を取る
    b = b - learning_rate * grad_b.mean(axis=0)  # 全てのデータセットについての平均値を取る

# 等高線プロット
plot_prediction(perceptron, w, b)

# データ点をプロット
plot_datapoint(x, t)


# 多層パーセプトロン(Multilayer perceptron)
単純パーセプトロンはXORゲートが表現できませんでした。この問題は層の数を増やす(多層にする)ことで解決することができます。単純パーセプトロンを多層にしたものを多層パーセプトロン(Multilayer perceptron)と呼びます。

グラフとして表すと、次の図のようになります。
<img src="jupyterFigure/MLP_graph.png" width="400">

数式で表すと、
$$
\begin{align*}
\mathbf{z}^{(1)} &= \sigma(\mathbf{w}^{(1)}\cdot \mathbf{x} + b^{(1)}) \\
y(\mathbf{x}|\mathbf{w}^{(1)}, b^{(1)},\mathbf{w}^{(2)}, b^{(2)}) &= \sigma(\mathbf{w}^{(2)}\cdot \mathbf{z}^{(1)} + b^{(2)})
\end{align*}
$$
のようになります。単純パーセプトロンの出力ノードを、新たに入力として使ったような形になっています。

活性化関数としてステップ関数を用いて、多層パーセプトロンがXORゲートを表現できることを示します。

## 例1: XORゲート
NANDゲート・ORゲート・ANDゲートは単純パーセプトロンで構成できることを先程示しました。
XORはNANDゲート・ORゲート・ANDゲートを組み合わせることで作ることができます。

NANDゲートの出力を$z_1$、ORゲートの出力を$z_2$とし、$z_1$、$z_2$を入力として受け取るANDゲートを考えると、表のようになります。

| $x_1$ | $x_2$ | $z_1$ | $z_2$ | $y$ |
| -- | -- | -- | -- | -- |
| 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 1 | 0 |

一例ですが、数式で明示的に書くと、
$$
\begin{align*}
z_1(x_1, x_2) &= \Gamma(-(x_1 + x_2 - 1.5)) \ \text{(NANDゲート)} \\
z_2(x_1, x_2) &= \Gamma(x_1 + x_2 - 0.5) \ \text{(ORゲート)} \\
y(z_1, z_2) &= \Gamma(z_1 + z_2 - 1.5) \ \text{(ANDゲート)}
\end{align*}
$$
のようにしてXORゲートを構成することができます。

この構成を視覚的に表したのが以下の図です。

|ORゲート|NANDゲート|ANDゲート|XORゲート|
|:---:|:---:|:---:|:---:|
|![](jupyterFigure/OR.png)|![](jupyterFigure/NAND.png)|![](jupyterFigure/AND.png)|![](jupyterFigure/XOR.png)||

１層のパーセプトロンでは、直線で領域分割することしかできませんでしたが、複数のパーセプトロンを組み合わせることでより複雑な領域の指定が可能になっていることがわかります。

In [ ]:
def xor_gate(x1, x2):
    # OR Gate
    a1 = or_gate(x1, x2)

    # NAND Gate
    a2 = nand_gate(x1, x2)

    # AND Gate
    y = and_gate(a1, a2)

    return y


# 正しく実装できているかを出力
print("XOR Gate:")
print("  x1, x2, y")
print("   0,  0, ", xor_gate(0, 0), "Correct" if xor_gate(0, 0) == 0 else "Wrong")  # 正解は 0
print("   0,  1, ", xor_gate(0, 1), "Correct" if xor_gate(0, 1) == 1 else "Wrong")  # 正解は 1
print("   1,  0, ", xor_gate(1, 0), "Correct" if xor_gate(1, 0) == 1 else "Wrong")  # 正解は 1
print("   1,  1, ", xor_gate(1, 1), "Correct" if xor_gate(1, 1) == 0 else "Wrong")  # 正解は 0


## 例2: 二次元ガウシアン
単純パーセプトロンでは分類できなかった以下のようなデータセット


In [ ]:
# データ点の取得
x, t = get_dataset_2()

# データ点をプロット
plot_datapoint(x, t)


を多層パーセプトロンで分類してみましょう。

数式としては、
$$
\begin{align*}
a_j^{(1)} &= w_{ij}^{(1)}\cdot x_{i} + b^{(1)}\\
z_j^{(1)} &= \sigma(a_j^{(1)}) \\
a_j^{(2)} &= w_{ij}^{(2)}\cdot x_{i} + b^{(2)}\\
y         &= \sigma(a^{(2)})
\end{align*}
$$
となります。

<img src="jupyterFigure/MLP_graph.png" width="400">

多層パーセプトロンでは、中間層のノードの数は自由に変えることができます。ここではノード数を3としてどのような予測がされるかを見てみましょう。


In [ ]:
# データ点の取得
x, t = get_dataset_2()

# 多層パーセプトロンの定義
def multilayerPerceptron(x, w1, b1, w2, b2):
    # 1層目
    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)

    # 2層目
    a2 = np.dot(z1, w2) + b2
    y = sigmoid(a2)

    return y


# パーセプトロンのパラメータ (正規分布で初期化)
w1 = np.random.randn(2, 3)  # 入力ノード数=2, 出力ノード数=3
b1 = np.random.randn(3)  # 出力ノード数=3
w2 = np.random.randn(3, 1)  # 入力ノード数=3, 出力ノード数=1
b2 = np.random.randn(1)  # 出力ノード数=1

# 多層パーセプトロンの出力をプロット
plot_prediction(multilayerPerceptron, w1, b1, w2, b2)

## データ点をプロット
plot_datapoint(x, t)


中間層のノードの数や、パラメータの値を変えてみて、どのように予測が変化するか見てみましょう。

In [ ]:
# データ点の取得
x, t = get_dataset_2()

# 多層パーセプトロンの定義
def multilayerPerceptron(x, w1, b1, w2, b2):
    # 1層目
    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)

    # 2層目
    a2 = np.dot(z1, w2) + b2
    y = sigmoid(a2)

    return y


# パーセプトロンのパラメータ
num_z = 3  # 中間層のノード数
w1 = np.zeros((2, num_z))  # 入力ノード数=2, 出力ノード数=num_z
b1 = np.zeros(num_z)  # 出力ノード数=num_z
w2 = np.zeros((num_z, 1))  # 入力ノード数=num_z, 出力ノード数=num_1
b2 = np.zeros(1)  # 出力ノード数=1

# FIXME: モデルパラメータの値を適当に変更してみましょう
# 1層目のモデルパラメータ
## z_0^{(1)}に関するパラメータ
w1[0][0] = -1.0  # 1層目の0番目のノードと2層目の0番目のノードをつなぐ重みを変更
w1[1][0] = +0.0  # 1層目の1番目のノードと2層目の0番目のノードをつなぐ重みを変更
b1   [0] = +3.0  # 2層目の0番目のノードのバイアスを変更

## z_1^{(1)}に関するパラメータ
w1[0][1] = +0.0  # 1層目の0番目のノードと2層目の1番目のノードをつなぐ重みを変更
w1[1][1] = +1.0  # 1層目の1番目のノードと2層目の1番目のノードをつなぐ重みを変更
b1   [1] = +2.0  # 2層目の1番目のノードのバイアスを変更

## z_2^{(1)}に関するパラメータ
w1[0][2] = +1.0  # 1層目の0番目のノードと2層目の2番目のノードをつなぐ重みを変更
w1[1][2] = +1.0  # 1層目の1番目のノードと2層目の2番目のノードをつなぐ重みを変更
b1   [2] = +2.0  # 2層目の2番目のノードのバイアスを変更

# 2層目のモデルパラメータ
## z_i^{(1)}から出力(y)を作るパラメータ
w2[0][0] = +1.0  # 2層目の0番目のノードと3層目の0番目のノードをつなぐ重みを変更
w2[1][0] = +1.0  # 2層目の1番目のノードと3層目の0番目のノードをつなぐ重みを変更
w2[2][0] = +1.0  # 2層目の2番目のノードと3層目の0番目のノードをつなぐ重みを変更
b2   [0] = -3.0  # 3層目の0番目のノードのバイアスを変更

# プロット
plot_prediction(multilayerPerceptron, w1, b1, w2, b2)

## データ点をプロット
plot_datapoint(x, t)


## 多層パーセプトロンの学習 (誤差逆伝搬)
先程の単純パーセプトロンでは、誤差関数の一次微分を手計算で求めました。
層の数を増やした多層パーセプトロンでは、一次微分を手計算するのは困難になります。
微分の計算を効率よく行える手法として誤差逆伝搬法があります。
ここでは、中間層が2層の多層パーセプトロンを例に、その手法について解説します。

![Test](jupyterFigure/MLP_3layer_graph.png "Tmp")

ここでは簡単のためバイアスの項を省略します(バイアスが入った場合についても簡単に拡張できます)。
ネットワークを表す式は、
$$
\begin{align*}
a_j^{(1)} &= w_{ij}^{(1)}\cdot x_{i}\\
z_j^{(1)} &= \sigma(a_j^{(1)}) \\
a_j^{(2)} &= w_{ij}^{(2)}\cdot z_{i}^{(1)}\\
z_j^{(2)} &= \sigma(a_j^{(2)}) \\
a_j^{(3)} &= w_{ij}^{(3)}\cdot z_{i}^{(2)}\\
y           &= \sigma(a^{(3)})
\end{align*}
$$
となります。

求めたいのは、誤差関数に対する各パラメータの一次微分です。すなわち
$\frac{\partial E}{\partial w_{ij}^{(1)}}$、$\frac{\partial E}{\partial w_{ij}^{(2)}}$、$\frac{\partial E}{\partial w_{ij}^{(3)}}$となります。

これらは連鎖率を使って求めることができます。
誤差関数は
$$
E(\mathbf{w}) = \frac{1}{N} \sum_{n=1}^{N} E_n(\mathbf{w}) = -\frac{1}{N} \sum_{n=1}^{N} \left( t_n \log y_n + \left( 1-t_n \right) \log \left( 1-y_n \right) \right)
$$
でした。
また、
$$
\delta_{i}^{(k)} = \frac{\partial E_n}{\partial a_{i}^{(k)}}
$$
として定義して、それぞれ計算をすると、

$$
\begin{align*}
\frac{\partial E_n}{\partial w_{ij}^{(3)}}
&= \frac{\partial E_n}{\partial  a_1^{(3)}}\cdot \frac{\partial  a_1^{(3)}}{\partial w_{ij}^{(3)}} \\
&= \left(y_n-t_n\right) \cdot z_1^{(2)} = \delta_{1}^{(3)} \cdot z_1^{(2)}
\end{align*}
$$

$$
\begin{align*}
\frac{\partial E_n}{\partial w_{ij}^{(2)}}
&= \frac{\partial E_n}{\partial  a_j^{(2)}}\cdot \frac{\partial  a_j^{(2)}}{\partial w_{ij}^{(2)}} \\
&= \left[ \frac{\partial E_n}{\partial  a_1^{(3)}}\cdot \frac{\partial a_1^{(3)}}{\partial  z_j^{(2)}}\cdot \frac{\partial z_j^{(2)}}{\partial  a_j^{(2)}}\right] \cdot \frac{\partial  a_j^{(2)}}{\partial w_{ij}^{(2)}} \\
&= \left[ \delta_{1}^{(3)}
\cdot w_{j1}^{(3)} 
\cdot \sigma^{'}(a_j^{(2)}) \right]
\cdot z_{i}^{(1)}  = \delta_{j}^{(2)} \cdot z_{i}^{(1)}
\end{align*}
$$

$$
\begin{align*}
\frac{\partial E_n}{\partial w_{ij}^{(1)}}
&= \frac{\partial E_n}{\partial  a_j^{(1)}}\cdot \frac{\partial  a_j^{(1)}}{\partial w_{ij}^{(1)}} \\
&= \left[ \sum_k \frac{\partial E_n}{\partial  a_{k}^{(2)}}\cdot \frac{\partial a_{k}^{(2)}}{\partial  z_j^{(1)}}\cdot \frac{\partial z_j^{(1)}}{\partial  a_j^{(1)}}\right] \cdot \frac{\partial  a_j^{(1)}}{\partial w_{ij}^{(1)}} \\
&= \left[ \sum_k \delta_{k}^{(2)}
\cdot w_{jk}^{(2)} 
\cdot \sigma^{'}(a_j^{(1)}) \right]
\cdot x_{i}  = \delta_{j}^{(1)} \cdot x_{i}
\end{align*}
$$

ここで、入力に近い方にある重みの計算では、それよりも後にある重みの計算に使用した値が再利用することができることがわかります。
このことを利用すると、複雑なネットワークの微分の計算がシステマティックに行えることになります。
![Test](jupyterFigure/MLP_3layer_bp.png "Tmp")


実際に計算する際には、下流側から順に$\delta_{i}^{(k)}$を求めていきます。
$\delta_{i}^{(k)}$は活性化関数(ここではシグモイド関数)の微分値に、下流の誤差($\delta_{j}^{(k+1)}$)の線形和をとったものとなります。
$$
\delta_{i}^{(k)} = 
\sigma^{'}(a_i^{(k)}) \left(
\sum_j w_{ij}^{(k+1)} \cdot \delta_{j}^{(k+1)} \right)
$$
重み($w_{ij}^{(k)}$)の微分はつながっているノードの$\delta_{i}^{(k)}$と$z_{i}^{(k)}$を用いて、
$$
\frac{\partial E_n}{\partial w_{ij}^{(k)}}  = \delta_{j}^{(k)} \cdot z_{i}^{(k-1)}
$$
として求めることができます。

このようにして微分を求めることを誤差逆伝搬(Back-propagation)と呼びます。

この誤差逆伝搬の式を先程の中間層1層の多層パーセプトロンに適用してみましょう。
<img src="jupyterFigure/MLP_1layer_bp.png" width="400">
次のような手順で一次微分を求めることができます。

1. 入力($x_i$)とパラメータ($w_{ij}^{(1)},b_j^{(1)},w_{ij}^{(2)},b_j^{(2)}$)を元に順伝搬させて予測値$y$を得る。
    $$
    \begin{align*}
    a_j^{(1)} &= w_{ij}^{(1)}\cdot x_{i} + b^{(1)}\\
    z_j^{(1)} &= \sigma(a_j^{(1)}) \\
    a_j^{(2)} &= w_{ij}^{(2)}\cdot x_{i} + b^{(2)}\\
    y         &= \sigma(a^{(2)})
    \end{align*} $$
2. 出力$y$と真のラベル$t$を用いて誤差$\delta$を伝搬させる。
    $$
    \begin{align*}
    \delta_{1}^{(2)} &= \left(y_n-t_n\right) \\
    \delta_{j}^{(1)} &= \sigma^{'}(a_j^{(1)}) \cdot w_{j1}^{(2)}  \cdot  \delta_{1}^{(2)} 
    \end{align*} $$
3. 誤差$\delta$を元に微分を計算
    $$
    \begin{align*}
    \frac{\partial E_n}{\partial w_{ij}^{(2)}} &= \delta_{j}^{(2)} \cdot z_{i}^{(1)} \\
    \frac{\partial E_n}{\partial b_{j}^{(2)}}  &= \delta_{j}^{(2)} \\
    \frac{\partial E_n}{\partial w_{ij}^{(1)}} &= \delta_{j}^{(1)} \cdot x_{i} \\
    \frac{\partial E_n}{\partial b_{j}^{(1)}}  &= \delta_{j}^{(1)} \\
    \end{align*} $$

誤差逆伝搬法をpythonで実装したものが以下になります。

In [ ]:

# シグモイド関数の微分
def grad_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))


# データ点の取得
x, t = get_dataset_2()

# パーセプトロンのパラメータ
w1 = np.random.randn(2, 3)  # 入力ノード数=2, 出力ノード数=3
b1 = np.random.randn(3)  # 出力ノード数=3
w2 = np.random.randn(3, 1)  # 入力ノード数=3, 出力ノード数=1
b2 = np.random.randn(1)  # 出力ノード数=1

# 最急降下法での学習
learning_rate = 1.0  # ステップ幅
num_steps = 2000  # 繰り返し回数
for _ in range(num_steps):
    # 順伝搬させる
    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    y = sigmoid(a2)  # パーセプトロンの出力

    # 一次微分を求めるために逆伝搬させる
    grad_a2 = y - t
    grad_w2 = np.einsum('ij,ik->ijk', z1, grad_a2)  # grad_w2 = z1 * grad_a2
    grad_b2 = 1. * grad_a2

    grad_a1 = grad_sigmoid(a1) * (grad_a2 * w2.T)
    grad_w1 = np.einsum('ij,ik->ijk', x, grad_a1)  # grad_w1 = x * grad_a1
    grad_b1 = 1. * grad_a1

    # パラメータの更新 (mean(axis=0): 全てのデータ点に対しての平均値を使う)
    w2 = w2 - learning_rate * grad_w2.mean(axis=0)
    b2 = b2 - learning_rate * grad_b2.mean(axis=0)
    w1 = w1 - learning_rate * grad_w1.mean(axis=0)
    b1 = b1 - learning_rate * grad_b1.mean(axis=0)

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction(multilayerPerceptron, w1, b1, w2, b2)

## データ点をプロット
plot_datapoint(x, t)


単純パーセプトロンでは、ある直線に対してどの程度離れているか、という指標でしか分類が行えていませんでしたが、多層パーセプトロンでは複数の直線からの距離を元に推定が行われています。

上の学習で得られたパラメータを使って、実際に中間層でどのような分類が行われているのかを見てみます。
図の$z_{i}^{(1)}$に対応するノードの出力をプロットします。

In [ ]:
# パーセプトロンの出力を等高線プロット
# 重みが(w1, b1)の1層パーセプトロンの出力を描画
plot_prediction_line(perceptron, w1[:, 0], b1[0])  # 中間層の0番目のノードの出力
plot_prediction_line(perceptron, w1[:, 1], b1[1])  # 中間層の1番目のノードの出力
plot_prediction_line(perceptron, w1[:, 2], b1[2])  # 中間層の2番目のノードの出力

# データ点をプロット
plot_datapoint(x, t)


このように、複数の(ここでは3つの)直線での分類を組み合わせて、2次元円形の領域を抜き出しています。
隠れ層の存在により、複雑な関数を近似できることがわかりました。

## 万能近似定理
ここまでは分類問題を考えてきました。ここでは回帰問題を考えて、多層パーセプトロンの表現能力を確認します。

分類問題では、パーセプトロンの出力を確率とみなすため、0~1の範囲を取るように、シグモイド関数を使いました。
回帰問題では任意の範囲の出力を取らせるようにします。よく使うのは、最後の活性化関数として線形関数(すなわち何もしない)を使うことです。
式として表すと、
$$
\begin{align*}
a_j^{(1)} &= w_{j}^{(1)}\cdot x + b^{(1)}\\
z_j^{(1)} &= \sigma(a_j^{(1)}) \\
a^{(2)} &= w_{i}^{(2)}\cdot z_{i} + b^{(2)}\\
y         &= a^{(2)}
\end{align*}
$$
となります。最後の行がシグモイド関数から、恒等写像に変わっています。グラフとして表すと、
<img src="jupyterFigure/MLP_regression_graph.png" width="400">
のようになります。

In [ ]:
# パーセプトロンの定義 (回帰)
def multilayerPerceptronRegression(x, w1, b1, w2, b2):
    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    y = a2  # linear activation
    return y


入力として1次元の関数を使ってその表現能力を見てみます

誤差関数としては予測値と真の値との差の2乗和(二乗和誤差(mean squared error))を使います。すなわち、パラメータが観測値を説明する確率は
$$
p(\mathbf{t}|\mathbf{w},b ) = \prod_{n=1}^{N} \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{(y_n-t_n)^2}{2\sigma^2}}
$$
であり、最小化する誤差関数は
$$
E(\mathbf{w}, b) = \frac{1}{N} \sum_{n=1}^{N} E_n(\mathbf{w}, b) = \frac{1}{N} \sum_{n=1}^{N} \frac{1}{2} \left( y_n - t_n \right)^2
$$
誤差関数の微分は
$$
\begin{align*}
\frac{\partial E_n}{\partial  a_1^{(k)}} 
&= \frac{\partial E_n}{\partial  y} \frac{\partial y}{\partial  a_1^{(k)}} \\
&= \left. y_n-t_n \right.
\end{align*}
$$
となり、シグモイド関数+クロスエントロピーのときと似たような式になります。

### 二次曲線

In [ ]:
num_sample = 20

# x=(-1, 1), t=x^2
x = np.linspace(-1, 1, num_sample)[:, np.newaxis]
t = x * x

# パーセプトロンのパラメータ
num_nodes = 2  # 中間層のノード数
w1 = np.random.randn(1, num_nodes)  # 入力ノード数=1, 出力ノード数=num_nodes
b1 = np.random.randn(num_nodes)  # 出力ノード数=num_nodes
w2 = np.random.randn(num_nodes, 1)  # 入力ノード数=num_nodes, 出力ノード数=1
b2 = np.random.randn(1)  # 出力ノード数=1

# うまく最適化されないときは以下で初期化してください
# w1 = np.array([[+2.79, -2.791]])
# b1 = np.array([[-2.80, -2.801]])
# w2 = np.array([[2.55], [2.551]])
# b2 = np.array([[-0.29]])

# 最急降下法での学習
learning_rate = 1.0  # ステップ幅
num_steps = 10000  # 繰り返し回数
for _ in range(num_steps):
    # 順伝搬させる
    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    y = a2  # パーセプトロンの出力

    # 一次微分を求めるために逆伝搬させる
    grad_a2 = y - t
    grad_w2 = np.einsum('ij,ik->ijk', z1, grad_a2)  # grad_w2 = z1 * grad_a2
    grad_b2 = 1. * grad_a2

    grad_a1 = grad_sigmoid(a1) * (grad_a2 * w2.T)
    grad_w1 = np.einsum('ij,ik->ijk', x, grad_a1)  # grad_w1 = x * grad_a1
    grad_b1 = 1. * grad_a1

    # パラメータの更新 (mean(axis=0): 全てのデータ点に対しての平均値を使う)
    w2 = w2 - learning_rate * grad_w2.mean(axis=0)
    b2 = b2 - learning_rate * grad_b2.mean(axis=0)
    w1 = w1 - learning_rate * grad_w1.mean(axis=0)
    b1 = b1 - learning_rate * grad_b1.mean(axis=0)


# プロット
plot_prediction_regression(x, t, multilayerPerceptronRegression, w1, b1, w2, b2)


黒点がデータ点、赤線が多層パーセプトロンによる予測値を表しています。
点線はパーセプトロンの中間層の各ノードにおける出力を表していて、点線の和が最後の出力(赤線)となります。

二次関数($y=x^2$)は中間層が2層の多層パーセプトロンでおおよそ近似できていることがわかります。

### Sin(x)

In [ ]:
num_sample = 20

# x=(-pi, pi), t=sin(x)
x = np.linspace(-np.pi, np.pi, num_sample)[:, np.newaxis]
t = np.sin(x)

# パーセプトロンのパラメータ
num_nodes = 4  # 中間層のノード数
w1 = np.random.randn(1, num_nodes)  # 入力ノード数=1, 出力ノード数=num_nodes
b1 = np.random.randn(num_nodes)  # 出力ノード数=num_nodes
w2 = np.random.randn(num_nodes, 1)  # 入力ノード数=num_nodes, 出力ノード数=1
b2 = np.random.randn(1)  # 出力ノード数=1

# うまく最適化されないときは以下で初期化してください
# w1 = np.array([[+1.4, -1.4, 1.6, -1.8]])
# b1 = np.array([[-4.2, -4.2, -0.8, -1.2]])
# w2 = np.array([[-3.0], [3.0], [2.0], [-1.3]])
# b2 = np.array([[-0.3]])

# 最急降下法での学習
learning_rate = 1.0  # ステップ幅
num_steps = 10000  # 繰り返し回数
for _ in range(num_steps):
    # 順伝搬させる
    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    y = a2  # パーセプトロンの出力

    # 一次微分を求めるために逆伝搬させる
    grad_a2 = y - t
    grad_w2 = np.einsum('ij,ik->ijk', z1, grad_a2)  # grad_w2 = z1 * grad_a2
    grad_b2 = 1. * grad_a2

    grad_a1 = grad_sigmoid(a1) * (grad_a2 * w2.T)
    grad_w1 = np.einsum('ij,ik->ijk', x, grad_a1)  # grad_w1 = x * grad_a1
    grad_b1 = 1. * grad_a1

    # パラメータの更新 (mean(axis=0): 全てのデータ点に対しての平均値を使う)
    w2 = w2 - learning_rate * grad_w2.mean(axis=0)
    b2 = b2 - learning_rate * grad_b2.mean(axis=0)
    w1 = w1 - learning_rate * grad_w1.mean(axis=0)
    b1 = b1 - learning_rate * grad_b1.mean(axis=0)

# プロット
plot_prediction_regression(x, t, multilayerPerceptronRegression, w1, b1, w2, b2)


sinカーブも中間層のノード数4の多層パーセプトロンで近似することができました。

### 矩形関数
より一般的な関数もノードの数を増やすことで任意の精度で近似できることが知られています。
ここでは直感的にそれを確かめてみます。

多層パーセプトロンは以下のような関数を近似できます。
$$
\begin{align*}
y(x) &= \begin{cases}
    1 & (-0.1 < a < 0.1)\\
    0 & (\text{otherwise})
  \end{cases}
\end{align*}
$$

In [ ]:
num_sample = 100

x = np.linspace(-1, 1, num_sample).reshape(-1, 1)
t = np.where(np.logical_and(-0.1 < x, x < 0.1), 1, 0)

# パーセプトロンのパラメータ
w1 = np.array([[200, 200]])
b1 = np.array([-20, +20])
w2 = np.array([[-1], [1]])
b2 = np.array([0.])

# プロット
plot_prediction_regression(x, t, multilayerPerceptronRegression, w1, b1, w2, b2)


この関数を複数用意することで任意の関数が近似できます。
<img src="jupyterFigure/rectangularFunction.png" width="400">

# Keras
はじめに述べたように、最近はDeep learningが簡単に扱えるツールが多々あります。
ここではKerasというパッケージを使ってこれまで行ってきた学習を行います。
KerasはtensorflowやTheanoといったDeep learningパッケージを簡単に扱うようにするためのラッパーで、実際には裏ではTensorflow等のライブラリが動いています。

Kerasの詳細な使い方については[公式ドキュメント](https://keras.io/ja/)も適宜参照してください。


多層パーセプトロンで2次元ガウシアン状のシグナルを分類する問題をKerasでも実装してみましょう。

In [ ]:
# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env TF_NUM_INTEROP_THREADS=1
%env TF_NUM_INTRAOP_THREADS=1

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# データ点の取得
x, t = get_dataset_2()

# モデルの定義
model = Sequential([
    Dense(3, activation='sigmoid', input_dim=2),  # ノード数が3の層を追加。活性化関数はシグモイド関数。
    Dense(1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
])

#  誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=1.0))

#  トレーニング
model.fit(
    x=x,
    y=t,
    batch_size=len(x),  # バッチサイズ。一回のステップで全てのデータを使うようにする。
    epochs=3000,  # 学習のステップ数
    verbose=0,  # 1とするとステップ毎に誤差関数の値などが表示される
)

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction(model.predict)

## データ点をプロット
plot_datapoint(x, t)


パーセプトロンの実装が簡潔に行えていることがわかります。

Kerasではモデルの変更も簡単に行えます。
下の例では、
- 1層あたりのノード数の変更
- レイヤー数の変更
- 活性化関数の変更
- 誤差関数の最適化手法の変更

を行っています。

Kerasは深層学習で有用と知られている多くのテクニックが既に実装され、簡単に使えるようになっています。
例えば[誤差関数](https://keras.io/ja/losses/)、[最適化手法](https://keras.io/ja/optimizers/)、[活性化関数](https://keras.io/ja/activations/)など、使える関数が日本語でもまとめられています。


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# データ点の取得
x, t = get_dataset_2()

# モデルの定義
model = Sequential([
    Dense(32, activation='relu', input_dim=2),  # ノード数が32の層を追加。活性化関数はrelu。
    Dense(16, activation='relu'),  # ノード数が16の層を追加。活性化関数はrelu。
    Dense(8, activation='relu'),  # ノード数が8の層を追加。活性化関数はrelu。
    Dense(1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
])

#  誤差関数としてクロスエントロピーを指定。最適化手法は"adam"
model.compile(loss='binary_crossentropy', optimizer='adam')

#  トレーニング
model.fit(
    x=x,
    y=t,
    batch_size=128,  # len(x), #  バッチサイズ。128個のデータセット毎にパラメータの更新を行う。
    epochs=500,  # 学習のステップ数
    verbose=0,
)

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction(model.predict)

## データ点をプロット
plot_datapoint(x, t)


# 最後に

実習課題として以下の2つを用意しました。
- パラメータ最適化手法
- 活性化関数とパラメータの初期化

それぞれ別のjupyter notebookを用意しています。もし興味があれば取り組んでください。